# Managing network

Is a class that allows you to define complex neural networks in Torch. Simply use this class as a descendant.

In [2]:
import torch
from torch import nn

## Sequential

You can use `torch.nn.Sequential` to combine multiple network layers into a sequential chain. Find out more in the [specific page](managing_network/sequential.ipynb).

---

The following cell demonstrates a basic example where a linear transformation is applied to the input, followed by a ReLU activation function.

In [ ]:
size = 3

sequential = torch.nn.Sequential(
    torch.nn.Linear(size, size, bias=False),
    torch.nn.ReLU()
)

X = torch.randn([3, 3])
sequential(X)

tensor([[0.0000, 0.0000, 0.8781],
        [0.4362, 0.0000, 0.7350],
        [0.0000, 0.0000, 1.1225]], grad_fn=<ReluBackward0>)

## Separate class

You can define a neural network as a separate class, which allows you to add custom logic for initialization or network-specific procedures. To create a network class, follow these rules:

- **Inherit from `torch.nn.Module`:** This establishes your class as a PyTorch module, providing access to its functionality.
- **Call `super().__init__()` in the constructor:** This initializes the base `nn.Module` class, ensuring proper setup.
- **Define a `forward` method:** This method implements the computational procedure of your network. It defines how input data flows through your layers to produce output. 

---

The following cell defines a set of Linear layers whose size is determined during class creation. The forward method standardizes the data before applying the network. 

In [ ]:
class ExampleNetwork(torch.nn.Module):
    def __init__(self, layers_number: int, neurons: int):

        super().__init__()

        self.network = torch.nn.Sequential(*[
            torch.nn.Linear(neurons, neurons)
            for i in range(layers_number)
        ])
    
    def forward(self, X: torch.Tensor):
        X = (X - X.mean(axis=0, keepdim=True))/X.std(axis=0, keepdim=True)
        return self.network(X)

Let's check if the network we've defined works as expected. 

In [ ]:
ExampleNetwork(layers_number=10, neurons=3)(X = torch.randn([5, 3]))

tensor([[-0.2482,  0.0882,  0.4507],
        [-0.2465,  0.0897,  0.4466],
        [-0.2531,  0.0827,  0.4587],
        [-0.2463,  0.0899,  0.4459],
        [-0.2461,  0.0892,  0.4429]], grad_fn=<AddmmBackward0>)

## `parameters()`

<a href="https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.parameters">Official documentation</a>.

To optimise the network, you need access to the parameters that will change as the model is optimised. The `Parameters` method fulfils this role. It looks like it somehow understands that the fields it contains are descendants of the class `nn.Module` and extracts their `parameters`.

Two following cells allow you to compare what `parameters` return if we use just empty ancestor of `nn.Module` and ancestor that have some fields that actually implementations of `nn.Module`.

In the following cell we have an empty `nn.Module` - so when we try to unpack it generator to list we have just an empty list:

In [45]:
class EmptyNetwork(nn.Module):
    pass
empty_network = EmptyNetwork()
[i for i in empty_network.parameters()]

[]

This cell implements such a descendant of the `nn.Module`, taking some parameters from its files. To be more specific, there are two fully connected layers defined here. So we end up with four tensors, two matrices for fully connected layers and their biases:

In [68]:
class ParametersNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.foo = nn.Linear(3, 3)
        self.bar = nn.Linear(5, 5)

network = ParametersNetwork()
for i in network.parameters():
    print(i.data)

tensor([[ 0.0302, -0.4218,  0.3589],
        [ 0.0500, -0.0864,  0.2156],
        [-0.5365,  0.4816,  0.2940]])
tensor([-0.0345, -0.4456, -0.4284])
tensor([[ 0.1643,  0.3919,  0.4324, -0.2549,  0.0276],
        [-0.0170,  0.2218, -0.3960,  0.1335,  0.3155],
        [ 0.0246, -0.0172,  0.3138,  0.4222,  0.0569],
        [ 0.2502,  0.3730, -0.1723,  0.1000,  0.1558],
        [ 0.2668, -0.3949,  0.1780,  0.2885,  0.2406]])
tensor([-0.0279, -0.2638,  0.1210, -0.1394, -0.0539])
